In [ ]:
# %% [markdown]
# # 城市功能区分析 - 数据探索
# 
# 作者: 张笔弈
# 日期: 2026-01-20

# %%
import sys
sys.path.append("..")

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from src.data_preprocessing.poi_processor import POIProcessor
import yaml

# %%
# 加载配置
with open("../config/settings.yaml", "r") as f:
    config = yaml.safe_load(f)

# %%
# 初始化处理器
processor = POIProcessor(config)

# %%
# 获取POI数据
poi_data = processor.fetch_poi_data("广州")

print(f"获取到 {len(poi_data)} 条POI记录")
print(f"POI类别分布:")
print(poi_data["type"].value_counts().head(10))

# %%
# 可视化POI分布
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# POI类别分布
poi_data["category"] = poi_data["type_code"].map({
    "050000": "餐饮", "060000": "购物", 
    "150000": "交通", "140000": "教育",
    "090000": "医疗", "120000": "办公"
})
category_counts = poi_data["category"].value_counts()

axes[0].bar(category_counts.index, category_counts.values)
axes[0].set_title("POI类别分布")
axes[0].tick_params(axis='x', rotation=45)

# 空间分布
axes[1].scatter(poi_data["lon"], poi_data["lat"], alpha=0.5, s=1)
axes[1].set_title("POI空间分布")
axes[1].set_xlabel("经度")
axes[1].set_ylabel("纬度")

plt.tight_layout()
plt.savefig("output/poi_distribution.png", dpi=300)
plt.show()

# %%
# 保存处理后的数据
clean_pois = processor.clean_poi_data(poi_data)
clean_pois.to_file("data/processed/guangzhou_pois.geojson", driver="GeoJSON")

print("数据探索完成!")